In [3]:
# This program aims to adjust the data to make it easier to be encoded with a DNA sequence while preserving the integrity of the data
# This program basically do two things:
# 1. To segment the hexdata with a block length between 48-64 digits, which corresponds to 96-128 bases after being converted to a DNA sequence. This is to keep the length of DNA sequences in accordance with next-gen sequencing options (typically 2*150 bp paired-end reading)  
# 2. To make sure each data segment contains at least one end symbol '20', which will be converted to a blank space in the decoding step. The '20' denotes the end of each segment. 


text_file = open("original hexdata-40.txt", "r")
my_hexdata = text_file.read()
text_file.close()

bitsNumber = len(my_hexdata)*4
print("Number of Bits: " + str(bitsNumber))
%store bitsNumber

chapteredData = []

while (my_hexdata != ''):
    for i in range (0, len(my_hexdata)):
        if my_hexdata[i:i+8] == '0D0A0D0A':         # '0D0A0D0A' is the symbol denoting the end of each chapter
            chapteredData.append(my_hexdata[0:i])
            my_hexdata = my_hexdata[(i+8):]
            
            break
    else:
        chapteredData.append(my_hexdata)
        
        break

chapteredDataBinary = []
scale = 16 ## equals to hexadecimal

num_of_bits = 8
segmentedChapteredData_i = []
segmentedChapteredData = []

for i in range (0, len(chapteredData)):           
    while (len(chapteredData[i]) >= 64):                               # Detect if a segment with a block length of 64 naturally ends with '20'
        if (chapteredData[i][62:64] == '20'):
            segmentedChapteredData_i.append(chapteredData[i][0:64])
            chapteredData[i] = chapteredData[i][64:]
            
        else:                                                          # If not, then detect the first '20' from the end to start
            for k in range (1, 32):
                if (chapteredData[i][62-2*k:64-2*k] == '20'):
                    list0 = list(chapteredData[i])
                    list0[64:] = list0[64-2*k:]
                    chapteredData[i] = ''.join(list0)
                    segmentedChapteredData_i.append(chapteredData[i][0:64-2*k])
                    chapteredData[i] = chapteredData[i][64:]
                    
                    break

    list1 = list(chapteredData[i])
    if (len(chapteredData[i]) < 48):
        list1[len(chapteredData[i]):48] = '20' * int((48-len(chapteredData[i]))/2)        # If the length of final data segment in one chapter is smaller than 48, then add multiple '20's to make it at least 48
    chapteredData[i] = ''.join(list1)

    segmentedChapteredData_i.append(chapteredData[i])
    
    for i in range (0, len(segmentedChapteredData_i)):     # To make the length of data segment a fold of 4
        if len(segmentedChapteredData_i[i])%4 != 0:
            segmentedChapteredData_i[i] = segmentedChapteredData_i[i] + '20'

        if len(segmentedChapteredData_i[i]) < 48:          # To make the length of data segment at least 48 by adding multiple '20's at the end 
            segmentedChapteredData_i[i] = segmentedChapteredData_i[i] + '20'*int((48-len(segmentedChapteredData_i[i]))/2) 
    
    segmentedChapteredData_i_joined = ''.join(segmentedChapteredData_i)

    segmentedChapteredData.append(segmentedChapteredData_i)
    segmentedChapteredData_i = []

%store segmentedChapteredData
    
for i in range (0, len(segmentedChapteredData)):
    segmentedChapteredData[i] = ''.join(segmentedChapteredData[i])

my_hexdata = ''.join(segmentedChapteredData)

print("Length of hex data: "+ str(len(my_hexdata)))
%store my_hexdata


Number of Bits: 338864
Stored 'bitsNumber' (int)
Stored 'segmentedChapteredData' (list)
Length of hex data: 87088
Stored 'my_hexdata' (str)
